This is the python notebook on implementing Sliding Window Attention

Importing landscape dataset

In [ ]:
import kagglehub
path = kagglehub.dataset_download("arnaud58/landscape-pictures")

Clong DiT repo and using model DiT_B_8

In [ ]:
!pip install torch torchvision timm accelerate einops scipy ftfy
!pip install torchmetrics
!git clone https://github.com/facebookresearch/DiT.git
%cd DiT
!pip install torch torchvision timm accelerate einops scipy ftfy torchmetrics
import sys
sys.path.append('.')

from models import DiT_B_8

In [ ]:
import os
for root, dirs, files in os.walk("/content"):
    if "models.py" in files:
        print("Found models.py at:", os.path.join(root, "models.py"))


Transforming and resizing the pictures

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import shutil

new_root = '/content/landscape_dataset/images'

os.makedirs(new_root, exist_ok=True)

for file in os.listdir(path):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        src = os.path.join(path, file)
        dst = os.path.join(new_root, file)
        shutil.copy(src, dst)

dataset_root = '/content/landscape_dataset'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3) ])

dataset = datasets.ImageFolder(root=dataset_root, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

real_img, _ = next(iter(dataloader))

In [ ]:
import torch
import numpy as np
import math
from timm.models.vision_transformer import PatchEmbed, Attention, Mlp
import torch.optim as optim
import torch.nn as nn
import shutil
from torch.optim.swa_utils import AveragedModel, update_bn

new_root = '/content/landscape_dataset/images'
dataset_root = '/content/landscape_dataset'

os.makedirs(new_root, exist_ok=True)


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3) ])

dataset = datasets.ImageFolder(root=dataset_root, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

real_img, _ = next(iter(dataloader))
fake_channel = torch.zeros((real_img.shape[0], 1, 256, 256))
real_img_4ch = torch.cat([real_img, fake_channel], dim=1)

def modulate(x, shift, scale):
    return x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)


class TimestepEmbedder(nn.Module):
    """
    Embeds scalar timesteps into vector representations.
    """
    def __init__(self, hidden_size, frequency_embedding_size=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size, bias=True),
            nn.SiLU(),
            nn.Linear(hidden_size, hidden_size, bias=True),
        )
        self.frequency_embedding_size = frequency_embedding_size

    @staticmethod
    def timestep_embedding(t, dim, max_period=10000):
        """
        Create sinusoidal timestep embeddings.
        :param t: a 1-D Tensor of N indices, one per batch element.
                          These may be fractional.
        :param dim: the dimension of the output.
        :param max_period: controls the minimum frequency of the embeddings.
        :return: an (N, D) Tensor of positional embeddings.
        """
        # https://github.com/openai/glide-text2im/blob/main/glide_text2im/nn.py
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
        ).to(device=t.device)
        args = t[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding

    def forward(self, t):
        t_freq = self.timestep_embedding(t, self.frequency_embedding_size)
        t_emb = self.mlp(t_freq)
        return t_emb


class LabelEmbedder(nn.Module):
    """
    Embeds class labels into vector representations. Also handles label dropout for classifier-free guidance.
    """
    def __init__(self, num_classes, hidden_size, dropout_prob):
        super().__init__()
        use_cfg_embedding = dropout_prob > 0
        self.embedding_table = nn.Embedding(num_classes + use_cfg_embedding, hidden_size)
        self.num_classes = num_classes
        self.dropout_prob = dropout_prob

    def token_drop(self, labels, force_drop_ids=None):
        """
        Drops labels to enable classifier-free guidance.
        """
        if force_drop_ids is None:
            drop_ids = torch.rand(labels.shape[0], device=labels.device) < self.dropout_prob
        else:
            drop_ids = force_drop_ids == 1
        labels = torch.where(drop_ids, self.num_classes, labels)
        return labels

    def forward(self, labels, train, force_drop_ids=None):
        use_dropout = self.dropout_prob > 0
        if (train and use_dropout) or (force_drop_ids is not None):
            labels = self.token_drop(labels, force_drop_ids)
        embeddings = self.embedding_table(labels)
        return embeddings



class SlidingWindowAttention(nn.Module):
    def __init__(self, dim, num_heads=8, window_size=8, qkv_bias=True):
        super().__init__()
        self.num_heads = num_heads
        self.window_size = window_size
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)  # (3, B, num_heads, N, dim)

        attn_scores = torch.zeros(B, self.num_heads, N, N, device=x.device)

        for i in range(N):
            start = max(0, i - self.window_size)
            end = min(N, i + self.window_size + 1)
            qi = q[:, :, i:i+1, :]  # (B, heads, 1, dim)
            ki = k[:, :, start:end, :]  # (B, heads, W, dim)
            attn_scores[:, :, i:i+1, start:end] = (qi @ ki.transpose(-2, -1)) * self.scale

        attn = attn_scores.softmax(dim=-1)
        out = attn @ v
        out = out.transpose(1, 2).reshape(B, N, C)
        return self.proj(out)

class DiTBlock(nn.Module):
    """
    A DiT block with adaptive layer norm zero (adaLN-Zero) conditioning.
    """
    def __init__(self, hidden_size, num_heads, mlp_ratio=4.0, **block_kwargs):
        super().__init__()
        self.norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        if block_kwargs.get("use_swa", False):
          self.attn = SlidingWindowAttention(hidden_size, num_heads=num_heads, window_size=block_kwargs.get("window_size", 5))
        else:
          self.attn = Attention(hidden_size, num_heads=num_heads, qkv_bias=True)

        self.norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        mlp_hidden_dim = int(hidden_size * mlp_ratio)
        approx_gelu = lambda: nn.GELU(approximate="tanh")
        self.mlp = Mlp(in_features=hidden_size, hidden_features=mlp_hidden_dim, act_layer=approx_gelu, drop=0)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 6 * hidden_size, bias=True)
        )

    def forward(self, x, c):
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = self.adaLN_modulation(c).chunk(6, dim=1)
        x = x + gate_msa.unsqueeze(1) * self.attn(modulate(self.norm1(x), shift_msa, scale_msa))
        x = x + gate_mlp.unsqueeze(1) * self.mlp(modulate(self.norm2(x), shift_mlp, scale_mlp))
        return x


class FinalLayer(nn.Module):
    """
    The final layer of DiT.
    """
    def __init__(self, hidden_size, patch_size, out_channels):
        super().__init__()
        self.norm_final = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.linear = nn.Linear(hidden_size, patch_size * patch_size * out_channels, bias=True)
        self.adaLN_modulation = nn.Sequential(
            nn.SiLU(),
            nn.Linear(hidden_size, 2 * hidden_size, bias=True)
        )

    def forward(self, x, c):
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)
        x = modulate(self.norm_final(x), shift, scale)
        x = self.linear(x)
        return x


class DiT(nn.Module):
    """
    Diffusion model with a Transformer backbone.
    """
    def __init__(
        self,
        input_size=32,
        patch_size=2,
        in_channels=4,
        hidden_size=1152,
        depth=28,
        num_heads=16,
        mlp_ratio=4.0,
        class_dropout_prob=0.1,
        num_classes=1000,
        learn_sigma=True,
        use_swa=False,
        window_size=5
    ):
        super().__init__()
        self.learn_sigma = learn_sigma
        self.in_channels = in_channels
        self.out_channels = in_channels * 2 if learn_sigma else in_channels
        self.patch_size = patch_size
        self.num_heads = num_heads

        self.x_embedder = PatchEmbed(input_size, patch_size, in_channels, hidden_size, bias=True)
        self.t_embedder = TimestepEmbedder(hidden_size)
        self.y_embedder = LabelEmbedder(num_classes, hidden_size, class_dropout_prob)
        num_patches = self.x_embedder.num_patches
        # Will use fixed sin-cos embedding:
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, hidden_size), requires_grad=False)

        self.blocks = nn.ModuleList([
            DiTBlock(hidden_size, num_heads, mlp_ratio=mlp_ratio,use_swa=use_swa, window_size=window_size) for _ in range(depth)
        ])
        self.final_layer = FinalLayer(hidden_size, patch_size, self.out_channels)
        self.initialize_weights()

    def initialize_weights(self):
        # Initialize transformer layers:
        def _basic_init(module):
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        self.apply(_basic_init)

        # Initialize (and freeze) pos_embed by sin-cos embedding:
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed.shape[-1], int(self.x_embedder.num_patches ** 0.5))
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        # Initialize patch_embed like nn.Linear (instead of nn.Conv2d):
        w = self.x_embedder.proj.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))
        nn.init.constant_(self.x_embedder.proj.bias, 0)

        # Initialize label embedding table:
        nn.init.normal_(self.y_embedder.embedding_table.weight, std=0.02)

        # Initialize timestep embedding MLP:
        nn.init.normal_(self.t_embedder.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder.mlp[2].weight, std=0.02)

        # Zero-out adaLN modulation layers in DiT blocks:
        for block in self.blocks:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)
            nn.init.constant_(block.adaLN_modulation[-1].bias, 0)

        # Zero-out output layers:
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].weight, 0)
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].bias, 0)
        nn.init.constant_(self.final_layer.linear.weight, 0)
        nn.init.constant_(self.final_layer.linear.bias, 0)

    def unpatchify(self, x):
        """
        x: (N, T, patch_size**2 * C)
        imgs: (N, H, W, C)
        """
        c = self.out_channels
        p = self.x_embedder.patch_size[0]
        h = w = int(x.shape[1] ** 0.5)
        assert h * w == x.shape[1]

        x = x.reshape(shape=(x.shape[0], h, w, p, p, c))
        x = torch.einsum('nhwpqc->nchpwq', x)
        imgs = x.reshape(shape=(x.shape[0], c, h * p, h * p))
        return imgs

    def forward(self, x, t, y):
        """
        Forward pass of DiT.
        x: (N, C, H, W) tensor of spatial inputs (images or latent representations of images)
        t: (N,) tensor of diffusion timesteps
        y: (N,) tensor of class labels
        """
        x = self.x_embedder(x) + self.pos_embed  # (N, T, D)  T = H * W / patch_size ** 2
        t = self.t_embedder(t)                   # (N, D)
        y = torch.zeros_like(t)
    # (N, D)
        c = t + y                                # (N, D)
        for block in self.blocks:
            x = block(x, c)                      # (N, T, D)
        x = self.final_layer(x, c)                # (N, T, patch_size ** 2 * out_channels)
        x = self.unpatchify(x)                   # (N, out_channels, H, W)
        return x

    def forward_with_cfg(self, x, t, y, cfg_scale):
        """
        Forward pass of DiT, but also batches the unconditional forward pass for classifier-free guidance.
        """
        # https://github.com/openai/glide-text2im/blob/main/notebooks/text2im.ipynb
        half = x[: len(x) // 2]
        combined = torch.cat([half, half], dim=0)
        model_out = self.forward(combined, t, y)
        # For exact reproducibility reasons, we apply classifier-free guidance on only
        # three channels by default. The standard approach to cfg applies it to all channels.
        # This can be done by uncommenting the following line and commenting-out the line following that.
        # eps, rest = model_out[:, :self.in_channels], model_out[:, self.in_channels:]
        eps, rest = model_out[:, :3], model_out[:, 3:]
        cond_eps, uncond_eps = torch.split(eps, len(eps) // 2, dim=0)
        half_eps = uncond_eps + cfg_scale * (cond_eps - uncond_eps)
        eps = torch.cat([half_eps, half_eps], dim=0)
        return torch.cat([eps, rest], dim=1)


# https://github.com/facebookresearch/mae/blob/main/util/pos_embed.py

def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False, extra_tokens=0):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token and extra_tokens > 0:
        pos_embed = np.concatenate([np.zeros([extra_tokens, embed_dim]), pos_embed], axis=0)
    return pos_embed


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1) # (H*W, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float64)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb

def plot_output_comparison(out_original, out_swa, title1="original Attention", title2="SWA"):
    import matplotlib.pyplot as plt
    from torchvision.utils import make_grid

    def img_tensor_to_numpy(img):
        img = (img.clamp(-1, 1) + 1) / 2
        img = make_grid(img[:, :3], nrow=1).permute(1, 2, 0).cpu().detach().numpy()
        return img

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img_tensor_to_numpy(out_original))
    axs[0].set_title(title1)
    axs[0].axis("off")
    axs[1].imshow(img_tensor_to_numpy(out_swa))
    axs[1].set_title(title2)
    axs[1].axis("off")
    plt.tight_layout()
    plt.show()


def DiT_XL_2(**kwargs):
    return DiT(depth=28, hidden_size=1152, patch_size=2, num_heads=16, **kwargs)

def DiT_XL_4(**kwargs):
    return DiT(depth=28, hidden_size=1152, patch_size=4, num_heads=16, **kwargs)

def DiT_XL_8(**kwargs):
    return DiT(depth=28, hidden_size=1152, patch_size=8, num_heads=16, **kwargs)

def DiT_L_2(**kwargs):
    return DiT(depth=24, hidden_size=1024, patch_size=2, num_heads=16, **kwargs)

def DiT_L_4(**kwargs):
    return DiT(depth=24, hidden_size=1024, patch_size=4, num_heads=16, **kwargs)

def DiT_L_8(**kwargs):
    return DiT(depth=24, hidden_size=1024, patch_size=8, num_heads=16, **kwargs)

def DiT_B_2(**kwargs):
    return DiT(depth=12, hidden_size=768, patch_size=2, num_heads=12, **kwargs)

def DiT_B_4(**kwargs):
    return DiT(depth=12, hidden_size=768, patch_size=4, num_heads=12, **kwargs)

def DiT_B_8(**kwargs):
    return DiT(depth=12, hidden_size=768, patch_size=8, num_heads=12, **kwargs)

def DiT_S_2(**kwargs):
    return DiT(depth=12, hidden_size=384, patch_size=2, num_heads=6, **kwargs)

def DiT_S_4(**kwargs):
    return DiT(depth=12, hidden_size=384, patch_size=4, num_heads=6, **kwargs)

def DiT_S_8(**kwargs):
    return DiT(depth=12, hidden_size=384, patch_size=8, num_heads=6, **kwargs)


DiT_models = {
    'DiT-XL/2': DiT_XL_2,  'DiT-XL/4': DiT_XL_4,  'DiT-XL/8': DiT_XL_8,
    'DiT-L/2':  DiT_L_2,   'DiT-L/4':  DiT_L_4,   'DiT-L/8':  DiT_L_8,
    'DiT-B/2':  DiT_B_2,   'DiT-B/4':  DiT_B_4,   'DiT-B/8':  DiT_B_8,
    'DiT-S/2':  DiT_S_2,   'DiT-S/4':  DiT_S_4,   'DiT-S/8':  DiT_S_8,
}


device = "cuda" if torch.cuda.is_available() else "cpu"


model_original = DiT_B_8(input_size=256, in_channels=4).to(device)
model_swa = AveragedModel(model_original).to(device)

real_img_4ch = real_img_4ch.to(device)
t = torch.tensor([1], dtype=torch.long, device=device)
y = torch.tensor([0], dtype=torch.long, device=device)

with torch.no_grad():
    out_original = model_original(real_img_4ch, t, y)
    out_swa  = model_swa(real_img_4ch, t, y)

real_img_4ch = real_img_4ch.to(device)
t = t.to(device)
y = y.to(device)
model_original = model_original.to(device)
model_swa = model_swa.to(device)

loss_fn = nn.MSELoss()
optimizer_original = optim.Adam(model_original.parameters(), lr=0.0003)
optimizer_swa  = optim.Adam(model_swa.parameters(),  lr=0.0003)

Importing FID

In [ ]:
!pip install torchmetrics
!pip install torch-fidelity
from torchmetrics.image.fid import FrechetInceptionDistance
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

Running Model for 35 epochs

In [ ]:
from torchmetrics.image.fid import FrechetInceptionDistance
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fid_original = FrechetInceptionDistance(feature=2048).to(device)
fid_swa = FrechetInceptionDistance(feature=2048).to(device)

def denormalize_to_uint8(tensor):
    tensor = (tensor * 0.5 + 0.5).clamp(0, 1)
    return (tensor * 255).to(torch.uint8)

fid_scores_original = []
fid_scores_swa = []

num_epochs = 35

for epoch in range(num_epochs):
    model_original.train()

    for i, (real_img, _) in enumerate(dataloader):
        if i >= 50:
          break

        real_img = real_img.to(device)
        fake_channel = torch.zeros((real_img.size(0), 1, 256, 256), device=device)
        real_img_4ch = torch.cat([real_img, fake_channel], dim=1)

        t = torch.randint(0, 1000, (real_img.size(0),), device=device)
        y = torch.zeros(real_img.size(0), dtype=torch.long, device=device)

        optimizer_original.zero_grad()
        out_original = model_original(real_img_4ch, t, y)
        loss_original = loss_fn(out_original[:, :3], real_img)
        loss_original.backward()
        optimizer_original.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}, Step {i}, Original Loss: {loss_original.item():.4f}")


    model_swa.update_parameters(model_original)


    model_original.eval()
    model_swa.eval()
    fid_original.reset()
    fid_swa.reset()

    update_bn(dataloader, model_swa)

    real_imgs = []
    for real_img, _ in dataloader:
        real_img = real_img.to(device)
        real_imgs.append(real_img)
        real_img_uint8 = denormalize_to_uint8(real_img).to(device)
        fid_original.update(real_img_uint8, real=True)
        fid_swa.update(real_img_uint8, real=True)
        if len(real_imgs) >= 50:
            break

    with torch.no_grad():
        for img in real_imgs:
            B = img.shape[0]
            z = torch.zeros((B, 1, 256, 256), device=device)
            x = torch.cat([img, z], dim=1)
            t = torch.randint(0, 1000, (B,), device=device)
            y = torch.zeros(B, dtype=torch.long, device=device)

            fake_original = model_original(x, t, y)
            fake_original_uint8 = denormalize_to_uint8(fake_original[:, :3]).to(device)
            fid_original.update(fake_original_uint8, real=False)

            fake_swa = model_swa(x, t, y)
            fake_swa_uint8 = denormalize_to_uint8(fake_swa[:, :3]).to(device)
            fid_swa.update(fake_swa_uint8, real=False)

    score_original = fid_original.compute().item()
    score_swa = fid_swa.compute().item()

    fid_scores_original.append(score_original)
    fid_scores_swa.append(score_swa)

    print(f"\nEpoch {epoch+1}: FID (original) score = {score_original:.2f}", end='')
    print(f", FID (SWA) score = {score_swa:.2f}\n")

    plot_output_comparison(fake_original, fake_swa)


epochs = list(range(1, num_epochs + 1))

print("\n\nFor FID score plot, the lower the better\n\n")

plt.figure(figsize=(8, 5))
plt.plot(epochs, fid_scores_original, label='Original Attention', marker='o')
plt.plot(epochs, fid_scores_swa, label='SWA', marker='s')
plt.xlabel("Epoch")
plt.ylabel("FID Score")
plt.title("FID Score vs Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
